**Jupyter Notebook: Analysis of data from the Review, Tenure and Promotion project**

*Dr. Juan Pablo Alperin, Scholarly Communications Lab*

------------------

This is the code used to analyze the data found in the following paper: 

Alperin, J., Muñoz Nieves, C., Schimanski, L., Fischman, G., Niles, M., and McKiernan, E. (2018). How significant are the public dimensions of faculty work in review, promotion, and tenure documents? *Humanities Commons [preprint]* [doi.org/10.17613/M6W950N3](https://doi.org/10.17613/M6W950N35)

The input file itself can be found here: 

Alperin, Juan Pablo; Muñoz Nieves, Carol; Schimanski, Lesley; McKiernan, Erin C.; Niles, Meredith T., 2018, "Terms and Concepts found in Tenure and Promotion Guidelines from the US and Canada", [doi.org/10.7910/DVN/VY4TJE](https://doi.org/10.7910/DVN/VY4TJE), *Harvard Dataverse*, V1, UNF:6:RD6O/u5UKt8wHog/sJUx9Q== [fileUNF] 

In [ ]:
import numpy as np
import pandas as pd

from IPython.display import display, HTML

percent_float = '{:.0f}%'.format
normal_float = '{:.0f}'.format
long_float = '{:.4f}'.format

# Save the outputs to an excel
writer = pd.ExcelWriter('RPT_summaries.xlsx')

In [ ]:
# Read the dataset. Available from: https://doi.org/10.7910/DVN/VY4TJE
df = pd.read_csv('RPT_final_dataset.csv')

In [ ]:
# Some columns are descriptors, the rest are categories of interest
descriptors = ['level', 'country', 'inst_type', 'inst_subtype', 'uni_name', 'label', 'discipline']
categories = [x for x in df.columns if x not in descriptors]
    
# make a list of the first four to do some combined analysis
output_types = [x for x in categories if 'output_type_' in x and 'generic' not in x]
output_types_traditional = [x for x in output_types if 'performances' not in x and 'exhibitions' not in x]

publication_venues = [x for x in categories if 'publication_venue_' in x ]

metrics = [x for x in categories if 'metrics_' in x]

peer_reviews = [x for x in categories if 'peer_review_' in x]

impact_factor = [x for x in categories if 'if_' in x]

In [ ]:
# Aggregate some metrics
df['any_metric'] = df[metrics].apply(lambda x: x.any(), axis=1)
df['any_output_type'] = df[output_types].apply(lambda x: x.any(), axis=1)
df['any_output_type_traditional'] = df[output_types].apply(lambda x: x.any(), axis=1)
df['any_publication_venue'] = df[publication_venues].apply(lambda x: x.any(), axis=1)
df['any_peer_review'] = df[peer_reviews].apply(lambda x: x.any(), axis=1)

# remove the metrics_ from the list now that they are integrated
# categories = list(set(categories).difference(metrics)) 
categories += ['any_metric', 'any_output_type', 'any_output_type_traditional', 'any_publication_venue', 'any_peer_review']

# Considering mentions of IF and of High Impact Journals together
df['any_impact_factor'] = df[['metrics_impact_factor', 'metrics_high_impact_journals']].apply(lambda x: x.any(), axis=1)
categories += ['any_impact_factor']

In [ ]:
# Drop any nulls, means we did not have document
# Currently only drops Harvard University institutional-level doc
df = df[~df[categories[0]].isnull()]

df = df[descriptors].join(df[categories].astype(bool))

In [ ]:
# Make a new DF for mentions in UN, AU, UN or AU, AU but not UN (aggregations)
uni_or_au = df.groupby('uni_name').agg(lambda x: x.any())  # UN or at least one AU
print(len(uni_or_au))

uni = df[df.level == 'UN'] # UN only
print(len(uni))

au = df[df.level == 'AU'] # AUs all (AU-level analysis)
print(len(au))

# at least one AU
au_uni = au.groupby('uni_name').agg(lambda x: x.any())  # returns 60 institutions
au_uni = uni[descriptors].set_index('uni_name').join(au_uni[categories]) # has all 129
print(len(au_uni))

# at least one AU and NOT at UN level
au_not_uni = uni.set_index('uni_name').join(au_uni[categories], lsuffix='_un', rsuffix='_au')
for cat in categories:
    au_not_uni[cat] = au_not_uni[cat+'_un'] & au_not_uni[cat+'_au']
    del au_not_uni[cat+'_un']
    del au_not_uni[cat+'_au']
print(len(au_not_uni))

In [ ]:
def summarize_categories(df, name = 'n', multiply=False):
    tmp = df[categories].sum() #.divide(len(df))
    tmp.name = name
    tmp_df = pd.DataFrame(tmp)

    if multiply:
        m=100
    else:
        m=1
    tmp_df[tmp.name + '_p'] = tmp.divide(len(df)/m)
    return tmp_df


In [ ]:
pd.options.display.float_format = normal_float

# Produce a sheet that has all the above
final = summarize_categories(uni_or_au, 'UN+AUs uni level').join(
        summarize_categories(uni, 'UN uni level')).join(
        summarize_categories(au_uni, 'AUs uni level')).join(
        summarize_categories(au_not_uni, 'AUs not UN, uni level')).join(
        summarize_categories(au, 'AUs au level'))
# final.multiply(100)
final.to_clipboard()

final.to_excel(writer, 'all')

In [ ]:
pd.options.display.float_format = normal_float
x = uni_or_au.groupby('inst_type')

print(x.size())
y = x.sum().transpose()
y.loc['N'] = x.size().tolist()
y = y.reindex(['N'] + sorted([c for c in y.index if c != 'N']))
cols = sorted(y.columns, reverse=True)

cols = sorted(y.columns, reverse=True)
print_cols = []
for c in cols: 
    p_col = "%s_p" % c
    y[p_col] = y[c]/x.size()[c]
    print_cols += [c, p_col]
# print(x.sum().transpose()['R1'].divide(2))
# x.sum().transpose().to_clipboard()
y[print_cols].to_clipboard()
print(y[sorted(cols, reverse=True)])
y[print_cols].to_excel(writer, "inst_type")

In [ ]:
pd.options.display.float_format = normal_float
x = uni_or_au[uni_or_au.inst_subtype.isin(['R1', 'R2', 'R3', 'U15'])].groupby('inst_subtype')
print(x.size())
y = x.sum().transpose()

y.loc['N'] = x.size().tolist()
y = y.reindex(['N'] + sorted([c for c in y.index if c != 'N']))

cols = sorted(y.columns, reverse=True)
print_cols = []
for c in cols: 
    p_col = "%s_p" % c
    y[p_col] = y[c]/x.size()[c]
    print_cols += [c, p_col]
# print(x.sum().transpose()['R1'].divide(2))
# x.sum().transpose().to_clipboard()
y[sorted(y.columns)].to_clipboard()
print(y[cols])
y[print_cols].to_excel(writer, "inst_subtype (R-types)")


In [ ]:
# Disciplines at the academic unit level
# Get only R-type

print("R-type only")
print("-----------")
x = au[au.inst_type == 'R-type'].groupby(['uni_name', 'discipline']).agg(lambda x: x.any())  # returns 60 institutions
x.index = x.index.droplevel(0)
x = x.reset_index().groupby('discipline')

print(x.size())
y = x.sum().transpose()
y.loc['N'] = x.size().tolist()
y = y.reindex(['N'] + sorted([c for c in y.index if c != 'N']))

cols = sorted(y.columns, reverse=True)
print_cols = []
for c in cols: 
    p_col = "%s_p" % c
    y[p_col] = y[c]/x.size()[c]
    print_cols += [c, p_col]
# print(x.sum().transpose()['R1'].divide(2))
# x.sum().transpose().to_clipboard()
y[print_cols].to_clipboard()
print(y[sorted(cols, reverse=True)])

y[print_cols].to_excel(writer, "disciplines (R-types)")

In [ ]:
# save these summaries
writer.save()

In [ ]:
# Use this to cross any two categoreis

pd.options.display.float_format = percent_float
cat1 = 'any_metric'
cat2 = 'public_and/or_community_engagement'

df = uni_or_au[uni_or_au.inst_type == 'R-type']
df2 = df[[cat1, cat2]]
df2 = pd.crosstab(df2[cat1], df2[cat2]) # .apply(lambda x: round(x*100/len(df2)))
df2.to_clipboard()
print(df2)
print('------------')
print("N=%s" % len(df))
print("%s == TRUE: %s; FALSE == %s" % (cat1, df[cat1].sum(), len(df)-df[cat1].sum()))
print("%s == TRUE: %s; FALSE == %s" % (cat2, df[cat2].sum(), len(df)-df[cat2].sum()))